Import the libraries needed

In [1]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
from datetime import datetime
import psutil
from datetime import datetime
import psutil

import numpy as np
import os
import pyodbc
pyodbc.drivers()

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 

C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### All the data used has been provisined in Get_The_Data.ipynb

In [2]:
pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

cutoff_date = '20191231'

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-07-19 18:51:20.821145
svmem(total=16756752384, available=4811382784, percent=71.3, used=11945369600, free=4811382784)


### Database connections and SQL to retrieve the various data needed.

In [4]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"


# 1. Get all switchable policies

In [5]:
# giving directory name
filename = input_path + '\Full_Population.csv'
df_Horizon = pd.read_csv(filename, delimiter=',',low_memory=False, header=0)

# print stats.
print(len(df_Horizon.index))
print(df_Horizon.dtypes)

now = datetime.now()
print(now)
print('==========================')

df_Horizon.head(5)

8411354
ContractNumber    object
ProdCat           object
ProdCode          object
PolicyStatus      object
CoverEndDt        object
SnapshotDt        object
SnapshotYrMth      int64
dtype: object
2024-07-19 18:51:48.619458


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2016-07-31,201607
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2016-07-31,201607
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2016-07-31,201607
3,6075850C,XRPP,XRPP_2005V1,I,2034-12-01,2016-07-31,201607
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2016-07-31,201607


## Filter to only take rows since 2020.

In [6]:
#df_Horizon['SnapshotDt'] = pd.to_datetime(df_Horizon['SnapshotDt'])
#df_Horizon['SnapshotDtStr'] = df_Horizon['SnapshotDt'].astype(str)
df_Horizon = df_Horizon[df_Horizon.SnapshotYrMth > 201912]

# print stats.
print(len(df_Horizon.index))
now = datetime.now()
print(now)
print('==========================')

print(df_Horizon.dtypes)
df_Horizon.head(5)

6012382
2024-07-19 18:51:49.336937
ContractNumber    object
ProdCat           object
ProdCode          object
PolicyStatus      object
CoverEndDt        object
SnapshotDt        object
SnapshotYrMth      int64
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth
2398972,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001
2398973,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001
2398974,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001
2398975,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001
2398976,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001


## Set the dates for historical switches.

In [7]:
df_Horizon['3_Months_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['3_Months_Ago'] = df_Horizon['3_Months_Ago'] + DateOffset(months=-3)
df_Horizon['3_Months_Ago'] = df_Horizon['3_Months_Ago'].dt.strftime('%Y%m%d')
df_Horizon["3_Months_Ago"] = df_Horizon['3_Months_Ago'].str.replace("-","").astype(int)

df_Horizon['6_Months_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['6_Months_Ago'] = df_Horizon['6_Months_Ago'] + DateOffset(months=-6)
df_Horizon['6_Months_Ago'] = df_Horizon['6_Months_Ago'].dt.strftime('%Y%m%d')
df_Horizon["6_Months_Ago"] = df_Horizon['6_Months_Ago'].str.replace("-","").astype(int)

df_Horizon['1_Year_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['1_Year_Ago'] = df_Horizon['1_Year_Ago'] + DateOffset(months=-12)
df_Horizon['1_Year_Ago'] = df_Horizon['1_Year_Ago'].dt.strftime('%Y%m%d')
df_Horizon["1_Year_Ago"] = df_Horizon['1_Year_Ago'].str.replace("-","").astype(int)

df_Horizon['2_Year_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['2_Year_Ago'] = df_Horizon['2_Year_Ago'] + DateOffset(months=-24)
df_Horizon['2_Year_Ago'] = df_Horizon['2_Year_Ago'].dt.strftime('%Y%m%d')
df_Horizon["2_Year_Ago"] = df_Horizon['2_Year_Ago'].str.replace("-","").astype(int)

df_Horizon['3_Year_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['3_Year_Ago'] = df_Horizon['3_Year_Ago'] + DateOffset(months=-36)
df_Horizon['3_Year_Ago'] = df_Horizon['3_Year_Ago'].dt.strftime('%Y%m%d')
df_Horizon["3_Year_Ago"] = df_Horizon['3_Year_Ago'].str.replace("-","").astype(int)

df_Horizon['4_Year_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['4_Year_Ago'] = df_Horizon['4_Year_Ago'] + DateOffset(months=-48)
df_Horizon['4_Year_Ago'] = df_Horizon['4_Year_Ago'].dt.strftime('%Y%m%d')
df_Horizon["4_Year_Ago"] = df_Horizon['4_Year_Ago'].str.replace("-","").astype(int)

df_Horizon['5_Year_Ago'] = pd.to_datetime(df_Horizon['SnapshotDt'])
df_Horizon['5_Year_Ago'] = df_Horizon['5_Year_Ago'] + DateOffset(months=-60)
df_Horizon['5_Year_Ago'] = df_Horizon['5_Year_Ago'].dt.strftime('%Y%m%d')
df_Horizon["5_Year_Ago"] = df_Horizon['5_Year_Ago'].str.replace("-","").astype(int)

# print stats.
print(len(df_Horizon.index))
now = datetime.now()
print(now)
print('==========================')

print(df_Horizon.dtypes)
df_Horizon.head(5)

6012382
2024-07-19 19:04:12.880158
ContractNumber    object
ProdCat           object
ProdCode          object
PolicyStatus      object
CoverEndDt        object
SnapshotDt        object
SnapshotYrMth      int64
3_Months_Ago       int32
6_Months_Ago       int32
1_Year_Ago         int32
2_Year_Ago         int32
3_Year_Ago         int32
4_Year_Ago         int32
5_Year_Ago         int32
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago
2398972,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131
2398973,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131
2398974,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131
2398975,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131
2398976,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131


# 2. Get all fund switch records.

In [8]:
# giving directory name
All_Switches = input_path + '\All_Switches.csv'
df_All_Switches = pd.read_csv(All_Switches, delimiter=',',low_memory=False, header=0)

# print stats.
print(len(df_All_Switches.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_All_Switches.dtypes)
df_All_Switches.head(5)

201805
2024-07-19 19:04:13.674440
svmem(total=16756752384, available=4041060352, percent=75.9, used=12715692032, free=4041060352)
index                       int64
POLICY_NUMBER              object
LEGACY_FUND_CODE           object
PRODUCT                    object
PROCESSING_DATE            object
EFFECTIVE_DATE             object
UNIT_TYPE                  object
CHARGE_TYPE                object
CASH_VALUE                float64
RISK_COMMENCEMENT_DATE     object
Servicing_Broker_Code      object
Issue_Date                 object
Cover_End_Date             object
Anniversary_Date           object
Source                     object
SOURCE                     object
dtype: object


,index,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source,SOURCE
0,0,6535239K,XMLEM,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
1,1,6535239K,XSK10,GNRP_2012V1,2019-01-02,2018-12-28,A,B,-5527.4800,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
2,2,6535239K,XSKN3,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
3,3,6562520M,XSDIV,GNRP_2012V1,2019-01-02,2018-12-27,A,B,10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,NaN,P
4,4,6562520M,XSEDD,GNRP_2012V1,2019-01-02,2018-12-27,A,B,-10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,NaN,P


## Filter to only take rows since 2020.

In [9]:
# Filter out old dates.

df_All_Switches['PROCESSING_DATE'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
df_All_Switches['ProcessingYrMth'] = df_All_Switches['PROCESSING_DATE'].dt.strftime('%Y%m')

df_All_Switches = df_All_Switches[df_All_Switches.ProcessingYrMth > '201912']

# print stats.
print(len(df_All_Switches.index))

now = datetime.now()
print(now)
print('==========================')

print(df_All_Switches.dtypes)
df_All_Switches.head(5)

185907
2024-07-19 19:04:16.349814
index                              int64
POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE                    object
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE            object
Servicing_Broker_Code             object
Issue_Date                        object
Cover_End_Date                    object
Anniversary_Date                  object
Source                            object
SOURCE                            object
ProcessingYrMth                   object
dtype: object


,index,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source,SOURCE,ProcessingYrMth
15898,15898,6266215L,XINAE,XRPP_2005V1,2020-01-02,2019-12-27,A,B,294.0896,2006-08-01,N605,2006-07-17,1900-01-01,1900-01-01,NaN,P,202001
15899,15899,6266215L,XINAE,XRPP_2005V1,2020-01-02,2019-12-27,A,B,0.4295,2006-08-01,N605,2006-07-17,1900-01-01,1900-01-01,NaN,P,202001
15900,15900,6266215L,XMLEM,XRPP_2005V1,2020-01-02,2019-12-27,A,B,0.4295,2006-08-01,N605,2006-07-17,1900-01-01,1900-01-01,NaN,P,202001
15901,15901,6266215L,XMLEM,XRPP_2005V1,2020-01-02,2019-12-27,A,B,294.0896,2006-08-01,N605,2006-07-17,1900-01-01,1900-01-01,NaN,P,202001
15902,15902,6266215L,XSDIV,XRPP_2005V1,2020-01-02,2019-12-27,A,B,294.0896,2006-08-01,N605,2006-07-17,1900-01-01,1900-01-01,NaN,P,202001


In [10]:


# Filter out old dates.
#df_All_Switches = df_All_Switches[df_All_Switches.Processing_INT > 20191231]

# Reorder the columns
df_All_Switches = df_All_Switches[['POLICY_NUMBER', 'PROCESSING_DATE','ProcessingYrMth']]

# print stats.
print(len(df_All_Switches.index))
# Set the processing date to a proper date and drop duplicates
#df_All_Switches = df_All_Switches.drop_duplicates(subset=['POLICY_NUMBER','PROCESSING_DATE','ProcessingYrMth','Processing_INT'], keep='first')
df_All_Switches = df_All_Switches.drop_duplicates(subset=['POLICY_NUMBER','ProcessingYrMth'], keep='first')


# print stats.
print(len(df_All_Switches.index))

now = datetime.now()
print(now)
print('==========================')

print(df_All_Switches.dtypes)
df_All_Switches.head(5)

185907
7242
2024-07-19 19:04:16.460415
POLICY_NUMBER              object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth            object
dtype: object


,POLICY_NUMBER,PROCESSING_DATE,ProcessingYrMth
15898,6266215L,2020-01-02,202001
15914,6300152A,2020-01-02,202001
15930,1113205#,2020-01-03,202001
15934,6493604M,2020-01-03,202001
15943,6527516E,2020-01-03,202001


# 3. Merge the full population with the switches to create the prediction dataset.

In [11]:
# Change to int
df_All_Switches["ProcessingYrMth"] = df_All_Switches["ProcessingYrMth"].astype(int)
df_Horizon_with_Switches = pd.merge(df_Horizon, df_All_Switches, left_on = ['ContractNumber','SnapshotYrMth'], right_on = ['POLICY_NUMBER','ProcessingYrMth'] , how = 'left') 

# Change the Processing Date to a flag to indicate a switch.
df_Horizon_with_Switches['Switch_Flag'] = ''
df_Horizon_with_Switches = df_Horizon_with_Switches.drop(['POLICY_NUMBER'],axis=1)

# If the sum is less than 2, ignore this as a positive booking and set it to zero. Also set the positive booking flag.
df_Horizon_with_Switches['Switch_Flag'] = np.where(pd.isna(df_Horizon_with_Switches['PROCESSING_DATE']),0,1)

# print stats.
print(len(df_Horizon_with_Switches.index))

print(df_Horizon_with_Switches.dtypes)
print(df_All_Switches.dtypes)
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_with_Switches['Switch_Flag'].value_counts())
df_Horizon_with_Switches.head(5)

6012382
ContractNumber             object
ProdCat                    object
ProdCode                   object
PolicyStatus               object
CoverEndDt                 object
SnapshotDt                 object
SnapshotYrMth               int64
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth           float64
Switch_Flag                 int32
dtype: object
POLICY_NUMBER              object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
dtype: object
2024-07-19 19:04:24.784604
svmem(total=16756752384, available=4301094912, percent=74.3, used=12455657472, free=4301094912)
0    6006459
1       5923
Name: Switch_Flag, dtype: int64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0


# Add the comparable past switch dates for comparison.

In [12]:
df_All_Switches['3_Months_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['3_Months_Ago'] = df_All_Switches['3_Months_Ago'] + DateOffset(months=-3)
df_All_Switches['3_Months_Ago'] = df_All_Switches['3_Months_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["3_Months_Ago"] = df_All_Switches['3_Months_Ago'].str.replace("-","").astype(int)

df_All_Switches['6_Months_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['6_Months_Ago'] = df_All_Switches['6_Months_Ago'] + DateOffset(months=-6)
df_All_Switches['6_Months_Ago'] = df_All_Switches['6_Months_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["6_Months_Ago"] = df_All_Switches['6_Months_Ago'].str.replace("-","").astype(int)

df_All_Switches['1_Year_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['1_Year_Ago'] = df_All_Switches['1_Year_Ago'] + DateOffset(months=-12)
df_All_Switches['1_Year_Ago'] = df_All_Switches['1_Year_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["1_Year_Ago"] = df_All_Switches['1_Year_Ago'].str.replace("-","").astype(int)

df_All_Switches['2_Year_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['2_Year_Ago'] = df_All_Switches['2_Year_Ago'] + DateOffset(months=-24)
df_All_Switches['2_Year_Ago'] = df_All_Switches['2_Year_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["2_Year_Ago"] = df_All_Switches['2_Year_Ago'].str.replace("-","").astype(int)

df_All_Switches['3_Year_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['3_Year_Ago'] = df_All_Switches['3_Year_Ago'] + DateOffset(months=-36)
df_All_Switches['3_Year_Ago'] = df_All_Switches['3_Year_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["3_Year_Ago"] = df_All_Switches['3_Year_Ago'].str.replace("-","").astype(int)

df_All_Switches['4_Year_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['4_Year_Ago'] = df_All_Switches['4_Year_Ago'] + DateOffset(months=-48)
df_All_Switches['4_Year_Ago'] = df_All_Switches['4_Year_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["4_Year_Ago"] = df_All_Switches['4_Year_Ago'].str.replace("-","").astype(int)

df_All_Switches['5_Year_Ago'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])
#df_All_Switches['5_Year_Ago'] = df_All_Switches['5_Year_Ago'] + DateOffset(months=-60)
df_All_Switches['5_Year_Ago'] = df_All_Switches['5_Year_Ago'].dt.strftime('%Y%m%d')
df_All_Switches["5_Year_Ago"] = df_All_Switches['5_Year_Ago'].str.replace("-","").astype(int)

# print stats.
print(len(df_All_Switches.index))
print(df_All_Switches.dtypes)
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
df_All_Switches.head(5)

7242
POLICY_NUMBER              object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
dtype: object
2024-07-19 19:04:25.820512
svmem(total=16756752384, available=4281626624, percent=74.4, used=12475125760, free=4281626624)


,POLICY_NUMBER,PROCESSING_DATE,ProcessingYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago
15898,6266215L,2020-01-02,202001,20200102,20200102,20200102,20200102,20200102,20200102,20200102
15914,6300152A,2020-01-02,202001,20200102,20200102,20200102,20200102,20200102,20200102,20200102
15930,1113205#,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103
15934,6493604M,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103
15943,6527516E,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103


In [13]:
print(df_Horizon_with_Switches['Switch_Flag'].value_counts())
df_Horizon_with_Switches.head(5)

0    6006459
1       5923
Name: Switch_Flag, dtype: int64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0


In [14]:
# Drop obsolete columns
#df_Horizon_with_Switches = df_Horizon_with_Switches.drop(['PROCESSING_DATE'],axis=1)
#df_Horizon_with_Switches = df_Horizon_with_Switches.drop(['ProcessingYrMth'],axis=1)

# print stats.
print(len(df_Horizon_with_Switches.index))

print(df_Horizon_with_Switches.dtypes)
print(df_All_Switches.dtypes)

print(df_Horizon_with_Switches['Switch_Flag'].value_counts())
df_Horizon_with_Switches.head(5)

6012382
ContractNumber             object
ProdCat                    object
ProdCode                   object
PolicyStatus               object
CoverEndDt                 object
SnapshotDt                 object
SnapshotYrMth               int64
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth           float64
Switch_Flag                 int32
dtype: object
POLICY_NUMBER              object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
dtype: object
0    6006459
1       5923
Name: Switch_Flag, d

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0


In [15]:
df_All_Switches.rename(columns = {'POLICY_NUMBER':'ContractNumber'}, inplace = True)

# print stats.
print(len(df_Horizon_with_Switches.index))

print(df_Horizon_with_Switches.dtypes)
print(df_All_Switches.dtypes)

print(df_Horizon_with_Switches['Switch_Flag'].value_counts())
df_Horizon_with_Switches.head(5)

6012382
ContractNumber             object
ProdCat                    object
ProdCode                   object
PolicyStatus               object
CoverEndDt                 object
SnapshotDt                 object
SnapshotYrMth               int64
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth           float64
Switch_Flag                 int32
dtype: object
ContractNumber             object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
dtype: object
0    6006459
1       5923
Name: Switch_Flag, d

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0


## Random check of some matched switches.

In [16]:
df_Horizon_with_Switches.loc[df_Horizon_with_Switches['ContractNumber'] == '6535239K']

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
25697,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
106793,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-02-29,202002,20191129,20190829,20190228,20180228,20170228,20160229,20150228,NaT,NaN,0
189060,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-03-31,202003,20191231,20190930,20190331,20180331,20170331,20160331,20150331,NaT,NaN,0
272531,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-04-30,202004,20200130,20191030,20190430,20180430,20170430,20160430,20150430,NaT,NaN,0
357010,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-05-31,202005,20200229,20191130,20190531,20180531,20170531,20160531,20150531,NaT,NaN,0
442507,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-06-30,202006,20200330,20191230,20190630,20180630,20170630,20160630,20150630,NaT,NaN,0
528977,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-07-31,202007,20200430,20200131,20190731,20180731,20170731,20160731,20150731,NaT,NaN,0
616615,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-08-31,202008,20200531,20200229,20190831,20180831,20170831,20160831,20150831,NaT,NaN,0
705210,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-09-30,202009,20200630,20200330,20190930,20180930,20170930,20160930,20150930,NaT,NaN,0
794963,6535239K,GNRP,GNRP_2012V1,I,2031-08-19,2020-10-31,202010,20200731,20200430,20191031,20181031,20171031,20161031,20151031,NaT,NaN,0


In [17]:
df_Horizon_with_Switches.dtypes

ContractNumber             object
ProdCat                    object
ProdCode                   object
PolicyStatus               object
CoverEndDt                 object
SnapshotDt                 object
SnapshotYrMth               int64
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth           float64
Switch_Flag                 int32
dtype: object

In [18]:
df_All_Switches.dtypes

ContractNumber             object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
dtype: object

In [19]:
df_All_Switches['SnapshotYrMth'] = pd.to_numeric(df_All_Switches['ProcessingYrMth'], errors="coerce").fillna(0).astype('int64')
df_All_Switches.dtypes

ContractNumber             object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
SnapshotYrMth               int64
dtype: object

In [20]:
df_Horizon_with_Switches.head(5)

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0


In [21]:
df_Horizon_with_Switches.tail(5)

,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag
6012377,8208292H,GNPP,GNPP_2023V1,I,2071-12-01,2024-06-30,202406,20240330,20231230,20230630,20220630,20210630,20200630,20190630,NaT,NaN,0
6012378,8208301V,GNPP,GNPP_2023V1,I,2041-04-01,2024-06-30,202406,20240330,20231230,20230630,20220630,20210630,20200630,20190630,NaT,NaN,0
6012379,8208305B,GNPP,GNPP_2023V1,I,2065-01-01,2024-06-30,202406,20240330,20231230,20230630,20220630,20210630,20200630,20190630,NaT,NaN,0
6012380,8208340P,GNPP,GNPP_2023V1,I,2070-02-01,2024-06-30,202406,20240330,20231230,20230630,20220630,20210630,20200630,20190630,NaT,NaN,0
6012381,8208345M,GNPP,GNPP_2023V1,I,2089-10-01,2024-06-30,202406,20240330,20231230,20230630,20220630,20210630,20200630,20190630,NaT,NaN,0


In [22]:
df_All_Switches.head(5)

,ContractNumber,PROCESSING_DATE,ProcessingYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,SnapshotYrMth
15898,6266215L,2020-01-02,202001,20200102,20200102,20200102,20200102,20200102,20200102,20200102,202001
15914,6300152A,2020-01-02,202001,20200102,20200102,20200102,20200102,20200102,20200102,20200102,202001
15930,1113205#,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103,202001
15934,6493604M,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103,202001
15943,6527516E,2020-01-03,202001,20200103,20200103,20200103,20200103,20200103,20200103,20200103,202001


In [23]:
# print stats.
print(len(df_Horizon_with_Switches.index))

6012382


## Set the flag for where a contract has some kind of switch.

In [24]:
df_Horizon_with_Switches['Any_Switch'] = df_Horizon_with_Switches.ContractNumber.isin(df_All_Switches.ContractNumber).astype(int)
#df_Horizon_with_Switches['Past_Switch'] = df_All_Switches['PROCESSING_DATE'].lt(df_Horizon_with_Switches['PROCESSING_DATE']) & df_Horizon_with_Switches.ContractNumber.isin(df_All_Switches.ContractNumber).astype(int)

print(df_Horizon_with_Switches['Any_Switch'].value_counts())
df_Horizon_with_Switches.head(5)

0    5760772
1     251610
Name: Any_Switch, dtype: int64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag,Any_Switch
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0


In [25]:
df_Horizon_with_Switches['Past_Switch'] = (((df_Horizon_with_Switches.ContractNumber).isin(df_All_Switches.ContractNumber).astype(int)) & ((df_All_Switches.SnapshotYrMth.astype(int)).le(df_Horizon_with_Switches.SnapshotYrMth).astype(int)))
#df_Horizon_with_Switches['Past_Switch'] = df_All_Switches['PROCESSING_DATE'].lt(df_Horizon_with_Switches['PROCESSING_DATE']) & df_Horizon_with_Switches.ContractNumber.isin(df_All_Switches.ContractNumber).astype(int)

print(df_Horizon_with_Switches['Past_Switch'].value_counts())
df_Horizon_with_Switches.head(5)

0    6012377
1          5
Name: Past_Switch, dtype: int64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth,3_Months_Ago,6_Months_Ago,1_Year_Ago,2_Year_Ago,3_Year_Ago,4_Year_Ago,5_Year_Ago,PROCESSING_DATE,ProcessingYrMth,Switch_Flag,Any_Switch,Past_Switch
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,202001,20191031,20190731,20190131,20180131,20170131,20160131,20150131,NaT,NaN,0,0,0


# No significant past fund switch activity indicating potential future switches so no need to proceed.

## Clean up the file.

In [26]:

df_Horizon_with_Switches = df_Horizon_with_Switches.drop(['SnapshotYrMth','3_Months_Ago','6_Months_Ago','1_Year_Ago','2_Year_Ago','3_Year_Ago','4_Year_Ago','5_Year_Ago','PROCESSING_DATE','ProcessingYrMth','Any_Switch','Past_Switch'],axis=1)
# print stats.
print(len(df_Horizon_with_Switches.index))

print(df_Horizon_with_Switches.dtypes)
print(df_All_Switches.dtypes)
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_with_Switches['Switch_Flag'].value_counts())
df_Horizon_with_Switches.head(5)

6012382
ContractNumber    object
ProdCat           object
ProdCode          object
PolicyStatus      object
CoverEndDt        object
SnapshotDt        object
Switch_Flag        int32
dtype: object
ContractNumber             object
PROCESSING_DATE    datetime64[ns]
ProcessingYrMth             int32
3_Months_Ago                int32
6_Months_Ago                int32
1_Year_Ago                  int32
2_Year_Ago                  int32
3_Year_Ago                  int32
4_Year_Ago                  int32
5_Year_Ago                  int32
SnapshotYrMth               int64
dtype: object
2024-07-19 19:04:30.851372
svmem(total=16756752384, available=3605504000, percent=78.5, used=13151248384, free=3605504000)
0    6006459
1       5923
Name: Switch_Flag, dtype: int64


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0


# Now write out the data to file and in another notebook, add the rest of the features that need to be synthesised because of PII.

In [27]:
# Write out to a comma separated values file.
filename = input_path + '\Full_Population_With_Switch_Flags.csv'
df_Horizon_with_Switches.to_csv(filename, encoding='utf-8', index=False)        

# Types and row count
print(df_Horizon_with_Switches.dtypes)
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
print(df_Horizon_with_Switches['Switch_Flag'].value_counts())

print(filename)
df_Horizon_with_Switches.head(5)

ContractNumber    object
ProdCat           object
ProdCode          object
PolicyStatus      object
CoverEndDt        object
SnapshotDt        object
Switch_Flag        int32
dtype: object
2024-07-19 19:05:15.620126
svmem(total=16756752384, available=3514732544, percent=79.0, used=13242019840, free=3514732544)
0    6006459
1       5923
Name: Switch_Flag, dtype: int64
.\data\Full_Population_With_Switch_Flags.csv


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,Switch_Flag
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2020-01-31,0
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2020-01-31,0
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2020-01-31,0
3,6075850C,XRPP,XRPP_2005V1,F,2034-12-01,2020-01-31,0
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2020-01-31,0


In [28]:
%who_ls DataFrame 

['df_All_Switches', 'df_Horizon', 'df_Horizon_with_Switches']

In [29]:
del df_All_Switches
del df_Horizon
del df_Horizon_with_Switches

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 19:06:35.295531
svmem(total=16756752384, available=3860537344, percent=77.0, used=12896215040, free=3860537344)
